<a href="https://colab.research.google.com/github/A00827038/ModelacionAgentes/blob/main/Reto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reto

En este modelo, diseñaremos un sistema multiagente necesario para simular una intersección controlada por señales de semáforos inteligentes.



## Reglas

Dado
1. Grid de 10x10 espacios.
2. 2 Semáforos
3. Se generan los carros en alguna de las posiciones A o B cada 4 frames. Tiene una variable de dirección cual valor es escogido de las opciones de la variable de dirección del carril. 
4. Agente carril, con estado 0, 1 y 2, representando si es jardín, calle, o intersección. Y una variable de dirección en los carrils con estado 1.
5. Agente carril con estado 2 está en 1 celda, representando la intersección. 


Realizar lo Siguiente
1. Mientras no haya un vehículo cercano, el semáforo estará en luz amarilla. 
2. Cuando un vehículo se acerque a la intersección, enviará un mensaje con el tiempo estimado de arribo (tiempo estimado de donde está hasta cruzar la intersección. 
3. El semáforo dará luz verde al semáforo más cercano (semáforo sincronizado) y establecerá un programa de luces a partir de ese punto para el resto de los vehículos. 

## Imports

Antes de empezar a crear el modelo del juego de la vida con multiagentes es necesario tener instalado los siguientes paquetes:
- `python`: asegúrense de usar la versión 3+.
- `mesa`: el framework de Python para el modelado de agentes.
- `numpy`: es una biblioteca de Python para el manejo de matrices, arreglos, manipulación matemática, lógica y mucho más.
- `matplotlib`: es una biblioteca para crear visualizaciones estáticas, animadas e interactivas en Python.

Para poder modelar el juego de la vida usando el framework de `mesa` es necesario importar dos clases: una para el modelo general, y otro para los agentes. 

In [ ]:
!pip3 install mesa

     |████████████████████████████████| 668 kB 5.1 MB/s 
     |████████████████████████████████| 60 kB 3.4 MB/s 


In [ ]:
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Debido a que necesitamos un solo agente por celda elegimos `SingleGrid` que fuerza un solo objeto por celda.
from mesa.space import MultiGrid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

## Crear el modelo

Antes que nada el presente modelo se encuentra basado en el [tutorial introductorio](https://mesa.readthedocs.io/en/master/tutorials/intro_tutorial.html). Lo modifiqué un poco para que funcionara para el presente problema pero en esencia es lo mismo.

In [ ]:
def get_grid(model):
    '''
    Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
    param model: El modelo del cual optener el grid. 
    '''
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for content in cell_content:
          if isinstance(content, carrilAgent):
            grid[x][y] = content.estado
          elif isinstance(content, semaforoAgent):
            grid[x][y] = 3
          else:
            4

    return grid


class cocheAgent(Agent):
    '''
    Un coche que se mueve con una dirección predeterminada.
    '''
    def __init__(self, unique_id, model, direccion):
        super().__init__(unique_id, model)
        
        

    def step(self):
       pass
     
          

    def advance(self):
       pass
       
      
class semaforoAgent(Agent):
    '''
    Representa a un semáforo.
    '''
    def __init__(self, unique_id, model, estado):
        super().__init__(unique_id, model)
        self.estado = estado

    def step(self):
       pass
     
    def advance(self):
       pass

class carrilAgent(Agent):
    '''
    Representa a una celda de tipo carril.
    '''
    def __init__(self, unique_id, model, estado, direccion):
        super().__init__(unique_id, model)
        self.estado = estado
        self.direccion = direccion
        
    def step(self):
      pass
     
    def advance(self):
      pass
       
      

            
class IntersectionModel(Model):
    '''
    Define el modelo de la intersección .
    '''
    def __init__(self, width, height):
        self.num_semaforos = 2
        self.contador = 4
        self.grid = MultiGrid(width, height, True)
        self.schedule = SimultaneousActivation(self)
       

        #Creamos el piso o carriles
        carrilesCont = 1;
        for (content, x, y) in self.grid.coord_iter(): 
          #Si estamos en posición de intersección
          if (x,y) == (6,5) or (x,y) == (5,5):
            direccion = None;
            carril = carrilAgent(carrilesCont, self, 2, direccion)
          #Else si estamos en posición de carril
          elif (x == 4 and y < 4 ) or y == 5:
            direccion = [(1,0),(0,1)];
            #print(carrilesCont)
            carril = carrilAgent(carrilesCont, self, 1, direccion)
          elif (x,y) == (5,4):
            direccion = [(1,0),(0,1)];
            carril = carrilAgent(carrilesCont, self, 1, direccion)
          #Else
          else:
            direccion = None;
            carril = carrilAgent(carrilesCont, self, 0, direccion)
          self.grid.place_agent(carril, (x,y))
          self.schedule.add(carril)
          carrilesCont += 1
        
       
        #Creamos los semáforos 
        semaforosCont = 1;
        for (content, x, y) in self.grid.coord_iter(): 
          #Si estamos en posición del semáforo
          if (x,y) == (4,4) or (x,y) == (6,4):
            semaforo = semaforoAgent(carrilesCont+semaforosCont, self, 0)
            self.grid.place_agent(semaforo, (x,y))
            self.schedule.add(semaforo)
          semaforosCont += 1
         
        
        # Aquí definimos con colector para obtener el grid completo. Aquí recompilamos la información.
        self.datacollector = DataCollector(
            model_reporters={"Grid": get_grid})
        
        self.cont = semaforosCont + carrilesCont;

    #Función que crea los carros cada 4 frames en las posiciones A, B, C o D
    def carros(model):
      lista_posiciones = [(5,0),(0,4)]
      posicion = model.random.choice(lista_posiciones)
      lista_direcciones_inter = [(1, 0)]
      direccion = lista_direcciones_inter
      coche = cocheAgent(model.cont, model, direccion)
      model.grid.place_agent(coche, posicion)
      model.schedule.add(coche)
      model.cont += 1

     
      
        

    def step(self):
        '''
        En cada paso el colector tomará la información que se definió y almacenará el grid para luego graficarlo.
        '''
        if self.contador == 4:
            self.carros()
        
        self.contador -= 1

        if self.contador == 0:
          self.contador = 4
        
        self.datacollector.collect(self)
        self.schedule.step()
        

A continuación corremos el modelo

In [ ]:
# Definimos el tamaño del Grid
WIDTH = 10
HEIGHT = 10

# Definimos el tiempo máximo en segundos
TIEMPO_MAX = float(input('Tiempo máximo de ejecución (segundos): '))

# Registramos el tiempo de inicio y corremos el modelo
start_time = time.time()
model = IntersectionModel(WIDTH, HEIGHT)

#Corre si el tiempo transcurrido es menor al tiempo max, todavía no están los stacks llenos, y hay cajas solas.
while((time.time() - start_time) < TIEMPO_MAX):
    model.step()

# Tiempo necesario hasta que todas las cajas estén en pilas de 5 (o se haya llegado al tiempo máximo)
print('Tiempo:', str(datetime.timedelta(seconds=(time.time() - start_time))))


Tiempo máximo de ejecución (segundos): 1
Tiempo: 0:00:01.000251


Obtenemos la información que almacenó el colector, este nos entregará un DataFrame de pandas que contiene toda la información.

In [ ]:
all_grid = model.datacollector.get_model_vars_dataframe()
print(all_grid.iloc[0][0])

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 1. 1. 1. 0. 1. 3. 0. 0. 0.]
 [0. 0. 0. 0. 1. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 3. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]


Graficamos la información usando `matplotlib`

In [ ]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap="Reds")

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [ ]:
anim

KeyboardInterrupt: ignored

## CUDA

El paso siguiente será modificar el modelo anterior para que funcione con CUDA

#Conclusiones


**Analysis**

Una estrategia que pudiera mejorar la eficiencia del programa, y como consequencia disminuir el tiempo dedicado y la cantidad de movimientos realizados por los robots es incrementar el número de cajas máximas por estante. Esto disminuiría el número de lugares (estantes) dónde los robots pueden dejar las cajas. En efecto, el tiempo dedicado y la cantidad de movimientos sería menor. Las siguientes simulaciones comprueban esta hipotesis. Como se puede observar, la simulación 1, que utiliza un programa donde el número máximo de cajas por estante es 5, tiene un tiempo de ordenamiento de 0.3 segundos y 1027 pasos. Mientras la simulación 2, que utiliza un programa donde el número máximo de cajas por estante es 10, tiene un tiempo de ordenamiento de 0.25 segundos y 764 pasos.


**Simulación 1**
* Width: 10
* Height: 10
* Numero de cajas: 33
* Tiempo máximo de ejecución (segundos): 20
* Tiempo de ordenamiento: 0:00:00.319362
* Número de movimientos realizados: 1027

**Simulación 2**
* Width: 10
* Height: 10
* Numero de cajas: 33
* Tiempo máximo de ejecución (segundos): 20
* Tiempo de ordenamiento: 0:00:00.248687
* Número de movimientos realizados: 764